In [1]:
from parse_deck import parse_event_to_deck

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import re
from tqdm import tqdm

import pandas as pd
from collections import OrderedDict

import json
import os

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def wait_loading_circle(timeout: int = 20):
    WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='sk-circle-container']")))
    

In [4]:
# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }
all_categories = ["ルギアVSTAR", "ミュウVMAX", "ジュラルドンVMAX", "ゾロア", "ギラティナVSTAR", "others"]

decks = {}
store_file_name = 'store.json'
if os.path.exists(store_file_name):
    with open(store_file_name, 'r') as f:
        decks = json.load(f)

store_code_list = []
for c in all_categories:
    if c not in decks: continue
    for d in decks[c]:
        store_code_list.append(d["deck_code"])

In [5]:
store_code_list

['fVkkVV-TdBvvq-FFVVkk',
 'yyMpyU-ZdRF4p-My2Xyp',
 'xD8x8c-wOI3oc-8c88x4',
 'g9Lngg-ANkV0Y-nLNngL',
 'V1FkVk-fpAyjo-kkfVVk',
 '8848Dc-eV0vOc-88cx4x',
 'kkk5wF-x8jg0V-FkkFVV',
 'HngLLL-ZVd05p-NnHinP',
 'wkVffk-uP0d67-fVVkkk',
 'QQQNQL-NPh4pH-nnLQgn',
 'LiLnLN-kvIn0K-gnnngN',
 '9nnLHn-j9zkHS-LnnNHL',
 'yMySMy-5N2GEW-pMMSyy',
 'NnHLNL-AyDMpZ-nngQgN',
 'EyRMMy-l2TS6F-UypSyX',
 'c8cxcY-2yjOcA-xx8888',
 'D8D88Y-hvMB3w-xx8Dax',
 'x8x8Y8-t7mBOv-x8D8c4',
 'Yx8xcx-NExZXB-888484',
 'kVkdVF-cpj1hw-kVVFkF',
 'nnHnLL-p5qhFt-nnQL9g',
 'yyM3yy-nJzaii-MM3MMM',
 '88c8xD-SICu56-8DxYYx',
 'MyMyEy-n0OHG8-py2MMp',
 'MyMyMM-Aaozpi-M2Mypp',
 '8xxJca-bDexp3-8Dx8x8',
 '8xcK8x-HOqkJl-x8Kx8x',
 'VF5kkV-MPKEQA-FFVkfk',
 'vvwVkk-izAoY1-VVVVkk',
 '8YD8cG-WGmX5m-x8cxc4',
 'LggLnn-wWtJX9-nP6LgQ',
 'MUypSp-wNPdPA-yyEyMy',
 'gNLLnn-HVWCSR-6LNgnL',
 'x8Gxx8-r3jkiZ-c88cxY',
 'LLg6L6-hbt464-nNnHnn',
 '8xKYDx-V39txe-888xJ4',
 'LiLQnL-AKC8VD-LnnnQn',
 'Mpyy2y-brK9U9-MMS3MM',
 'yESyyy-P0KiMZ-MMypM3',
 'nQnLnL-np5wdd-LLgn69',


In [6]:
# parse CL event links from official website
url = "https://players.pokemon-card.com/event/result/list"
driver = webdriver.Chrome(options=chrome_options)  # options=chrome_options
driver.implicitly_wait(10) # seconds
driver.get(url)

page_limit = 10
event_limit = 100
page_cnt = 0
event_cnt = 0

while 1:
    events = driver.find_elements(By.CLASS_NAME, "eventListItem")
    for event in tqdm(events):
        title = event.find_element(By.CLASS_NAME, "title")
        if "シティリーグ" in title.text:
            num_people_str = event.find_element(By.CLASS_NAME, "capacity").text
            num_people = re.findall(r'\d+', num_people_str)
            num_people = int(num_people[0]) if len(num_people) == 1 else None
            event_link = event.get_attribute("href")
            parse_event_to_deck(event_link, num_people, decks, all_categories, skip_codes=store_code_list)
            event_cnt += 1
    page_cnt += 1

    if page_cnt >= page_limit or event_cnt >= event_limit:
        break

    # nevigate to the next page
    driver.find_element(By.CLASS_NAME, "btn.next").click()
    wait_loading_circle()

driver.close()

with open(store_file_name, 'w') as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

100%|███████████████████████████████████████████| 20/20 [01:30<00:00,  4.53s/it]


In [7]:
target_category = "ゾロア" # "ゾロア", "ギラティナVSTAR"
df_list = []

for card_type in ["pokemons", "tools", "supporters", "stages", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[target_category]):
        deck_code = deck["deck_code"]  # row id
        pokecard = OrderedDict()
        pokecard["date"] = deck["date"]
        pokecard["num_people"] = deck["num_people"]
        pokecard["rank"] = deck["rank"]
        pokecard.update(deck[card_type])
        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # select cols for analysis
    col_list = list(df)
    col_list.remove("date")
    col_list.remove("num_people")
    col_list.remove("rank")

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis='rows', numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis='rows')
    
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder rows in df
    num_rows = df.shape[0]
    target_rows = [num_rows-1, num_rows-2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]
    
    # sort by pick rate
    df = df.sort_values('pick_rate', axis=1, ascending=False)
    col_list = list(df)
    col_list.remove("date")
    col_list.remove("num_people")
    col_list.remove("rank")
    df = df[["date", "num_people", "rank"] + col_list]
    
    # store
    df_list.append(df)

In [8]:
df_list[0]

,date,num_people,rank,ゾロア,ゾロアーク,マナフィ,グラエナ,ライチュウ,タルップル,ヤドラン,テールナー,チラーミィ,チラチーノ,バサギリ,アップリュー,かがやくジラーチ,ヒスイ ゾロアーク,ミノマダム,ビーダル,ビッパ,ヒスイ ウインディ,チルタリス,ヒスイ ガーディ,ヤレユータン,チルット,マルマイン,ガラル ニャース,かがやくルチャブル
pick_rate,NaN,NaN,NaN,1.000000,1.000000,1.0,1.0,0.933333,0.933333,0.933333,0.8,0.733333,0.733333,0.733333,0.533333,0.533333,0.333333,0.266667,0.266667,0.266667,0.200000,0.133333,0.133333,0.133333,0.066667,0.066667,0.066667,0.066667
avg_num_used,NaN,NaN,NaN,3.933333,3.933333,1.2,1.0,1.000000,1.000000,1.000000,1.0,3.181818,2.909091,1.000000,1.000000,1.000000,1.000000,1.000000,1.750000,1.500000,2.666667,1.000000,2.500000,1.000000,1.000000,1.000000,1.000000,1.000000
LnQLQn-FLn7fe-nLnnNn,2022年11月23日(水),48.0,5.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,3.000000,3.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
yyyM2M-Zgfdkb-SME22M,2022年11月20日(日),32.0,5.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,3.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2MyMyy-c1qVpR-Sy3MEM,2022年11月20日(日),48.0,2.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,4.000000,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RMXyM3-PaMcu0-yMypM2,2022年11月20日(日),48.0,5.0,4.000000,4.000000,2.0,1.0,1.000000,1.000000,1.000000,1.0,4.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
88xxc8-UAuRpB-8Yxc4Y,2022年11月20日(日),76.0,5.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XRMpyp-lHu00T-MMyMRy,2022年11月20日(日),32.0,1.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,1.0,3.000000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VVFkkV-HetqHG-1dvF5k,2022年11月20日(日),34.0,5.0,4.000000,4.000000,2.0,1.0,1.000000,1.000000,1.000000,0.0,3.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
kkkkdw-bWIMXK-VVvFVw,2022年11月20日(日),32.0,5.0,4.000000,4.000000,1.0,1.0,1.000000,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000


In [9]:
df_list[1]

,date,num_people,rank,しんかのおこう,ふつうのつりざお,レスキューキャリー,レベルボール,ハイパーボール,クイックボール,こだわりベルト,やまびこホーン,バトルVIPパス,ふうせん,ヒスイのヘビーボール,ともだちてちょう,ロストスイーパー,あなぬけのヒモ,ポケモンいれかえ,そっくりベル,ポケギア3.0,ポケモンキャッチャー,回収ネット
pick_rate,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,0.733333,0.533333,0.400000,0.333333,0.333333,0.266667,0.266667,0.2,0.133333,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667
avg_num_used,NaN,NaN,NaN,3.466667,1.933333,1.466667,3.733333,2.818182,3.250000,1.166667,1.000000,3.400000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000
LnQLQn-FLn7fe-nLnnNn,2022年11月23日(水),48.0,5.0,3.000000,2.000000,1.000000,4.000000,3.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
yyyM2M-Zgfdkb-SME22M,2022年11月20日(日),32.0,5.0,4.000000,2.000000,2.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2MyMyy-c1qVpR-Sy3MEM,2022年11月20日(日),48.0,2.0,3.000000,1.000000,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RMXyM3-PaMcu0-yMypM2,2022年11月20日(日),48.0,5.0,4.000000,2.000000,2.000000,4.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
88xxc8-UAuRpB-8Yxc4Y,2022年11月20日(日),76.0,5.0,4.000000,2.000000,1.000000,3.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XRMpyp-lHu00T-MMyMRy,2022年11月20日(日),32.0,1.0,3.000000,2.000000,1.000000,4.000000,3.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VVFkkV-HetqHG-1dvF5k,2022年11月20日(日),34.0,5.0,3.000000,2.000000,2.000000,4.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
kkkkdw-bWIMXK-VVvFVw,2022年11月20日(日),32.0,5.0,4.000000,3.000000,1.000000,3.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
df_list[2]

,date,num_people,rank,セレナ,博士の研究,マリィ,ボスの指令,ザクロ,シロナの覇気,ピオニー,ヒガナの決意,ユウリ,クララ,オニオン,さぎょういん,ハマナのバックアップ,ボールガイ,とりつかい,ツツジ,フウロ
pick_rate,NaN,NaN,NaN,0.933333,0.733333,0.666667,0.666667,0.266667,0.133333,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
avg_num_used,NaN,NaN,NaN,3.785714,2.818182,2.000000,1.100000,1.000000,1.500000,3.500000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
LnQLQn-FLn7fe-nLnnNn,2022年11月23日(水),48.0,5.0,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
yyyM2M-Zgfdkb-SME22M,2022年11月20日(日),32.0,5.0,4.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2MyMyy-c1qVpR-Sy3MEM,2022年11月20日(日),48.0,2.0,4.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RMXyM3-PaMcu0-yMypM2,2022年11月20日(日),48.0,5.0,4.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
88xxc8-UAuRpB-8Yxc4Y,2022年11月20日(日),76.0,5.0,4.000000,4.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
XRMpyp-lHu00T-MMyMRy,2022年11月20日(日),32.0,1.0,4.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VVFkkV-HetqHG-1dvF5k,2022年11月20日(日),34.0,5.0,3.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
kkkkdw-bWIMXK-VVvFVw,2022年11月20日(日),32.0,5.0,2.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000


In [11]:
df_list[3]

,date,num_people,rank,頂への雪道,ポケストップ,未開の祭壇
pick_rate,NaN,NaN,NaN,0.666667,0.200000,0.066667
avg_num_used,NaN,NaN,NaN,1.800000,1.666667,2.000000
LnQLQn-FLn7fe-nLnnNn,2022年11月23日(水),48.0,5.0,2.000000,0.000000,0.000000
yyyM2M-Zgfdkb-SME22M,2022年11月20日(日),32.0,5.0,3.000000,0.000000,0.000000
2MyMyy-c1qVpR-Sy3MEM,2022年11月20日(日),48.0,2.0,1.000000,0.000000,0.000000
RMXyM3-PaMcu0-yMypM2,2022年11月20日(日),48.0,5.0,0.000000,0.000000,2.000000
88xxc8-UAuRpB-8Yxc4Y,2022年11月20日(日),76.0,5.0,1.000000,0.000000,0.000000
XRMpyp-lHu00T-MMyMRy,2022年11月20日(日),32.0,1.0,2.000000,0.000000,0.000000
VVFkkV-HetqHG-1dvF5k,2022年11月20日(日),34.0,5.0,0.000000,0.000000,0.000000
kkkkdw-bWIMXK-VVvFVw,2022年11月20日(日),32.0,5.0,1.000000,0.000000,0.000000


In [12]:
df_list[4]

,date,num_people,rank,キャプチャーエネルギー,ツインエネルギー,ダブルターボエネルギー,ギフトエネルギー,基本超エネルギー
pick_rate,NaN,NaN,NaN,0.933333,0.933333,0.800000,0.066667,0.066667
avg_num_used,NaN,NaN,NaN,2.857143,4.000000,2.166667,1.000000,2.000000
LnQLQn-FLn7fe-nLnnNn,2022年11月23日(水),48.0,5.0,3.000000,4.000000,2.000000,1.000000,0.000000
yyyM2M-Zgfdkb-SME22M,2022年11月20日(日),32.0,5.0,3.000000,4.000000,3.000000,0.000000,0.000000
2MyMyy-c1qVpR-Sy3MEM,2022年11月20日(日),48.0,2.0,2.000000,4.000000,1.000000,0.000000,2.000000
RMXyM3-PaMcu0-yMypM2,2022年11月20日(日),48.0,5.0,4.000000,4.000000,1.000000,0.000000,0.000000
88xxc8-UAuRpB-8Yxc4Y,2022年11月20日(日),76.0,5.0,4.000000,4.000000,1.000000,0.000000,0.000000
XRMpyp-lHu00T-MMyMRy,2022年11月20日(日),32.0,1.0,2.000000,4.000000,4.000000,0.000000,0.000000
VVFkkV-HetqHG-1dvF5k,2022年11月20日(日),34.0,5.0,4.000000,4.000000,2.000000,0.000000,0.000000
kkkkdw-bWIMXK-VVvFVw,2022年11月20日(日),32.0,5.0,3.000000,4.000000,0.000000,0.000000,0.000000


In [13]:
decks["others"]

[{'deck_link': 'https://www.pokemon-card.com/deck/confirm.html/deckID/wwVVkk-iAG7EA-kFvFfV',
  'deck_code': 'wwVVkk-iAG7EA-kFvFfV',
  'pokemons': {'ムゲンダイナVMAX': 3,
   'ムゲンダイナV': 3,
   'クロバットV': 2,
   'クロバットVMAX': 1,
   'ドラピオンV': 2,
   'ガラル マタドガス': 2,
   'ドガース': 3,
   'かがやくヒスイ オオニューラ': 1,
   'ガラル ジグザグマ': 2},
  'tools': {'クイックボール': 4,
   'ハイパーボール': 3,
   'あなぬけのヒモ': 4,
   '森の封印石': 1,
   'ダークパッチ': 2,
   'こだわりベルト': 2},
  'supporters': {'マリィ': 3, '博士の研究': 4, 'ボスの指令': 1, 'セレナ': 2},
  'stages': {'ガラル鉱山': 3},
  'energies': {'ハイド悪エネルギー': 3, '基本悪エネルギー': 6, 'Vガードエネルギー': 1},
  'rank': 3,
  'num_people': 48,
  'date': '2022年11月23日(水)'},
 {'deck_link': 'https://www.pokemon-card.com/deck/confirm.html/deckID/VVffVk-FqWupm-kkFwfF',
  'deck_code': 'VVffVk-FqWupm-kkFwfF',
  'pokemons': {'レジギガス': 2,
   'レジアイス': 2,
   'レジロック': 2,
   'レジスチル': 2,
   'レジドラゴ': 2,
   'レジエレキ': 1},
  'tools': {'クイックボール': 4,
   'ハイパーボール': 2,
   'ヒスイのヘビーボール': 1,
   'ふつうのつりざお': 4,
   'トレッキングシューズ': 4,
   '回収ネット': 4,
   'こだわりベルト': 2},
